In [1]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [13]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,study room,servant room,store room,furnishing_type,luxaury_category,floor_category,price
0,0.0,57.0,3,3,3.0,0.0,2178.0,0,0,0,0,2.0,2.0,1.69
1,0.0,111.0,2,2,3.0,1.0,1368.0,1,0,0,0,2.0,2.0,1.10
2,0.0,56.0,3,5,4.0,0.0,3200.0,0,1,0,2,1.0,2.0,6.25
3,0.0,94.0,3,3,4.0,3.0,1550.0,1,1,0,1,1.0,0.0,2.35
4,1.0,49.0,3,2,1.0,0.0,783.0,0,0,0,0,1.0,1.0,0.67


In [4]:
# applying OnehotEncoding on all categorical columns (sector,balcony,agePossession,furnishing_type,luxaury_category,floor_category)

In [14]:
X = df.drop(columns=['price'])
y = df['price']

In [15]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [20]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxaury_category', 'floor_category']

In [21]:
# Applying Log Transformation on y

y_transformed = np.log1p(y)

In [27]:
# columntransformer for preprocessing 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [28]:
# creating a pipeline

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [29]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [30]:
scores.mean()

0.8850295977207324

In [32]:
scores.std()

0.01311380548629832

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [34]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxaury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [35]:
y_pred = pipeline.predict(X_test)

C:\Users\OM CHAVAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [36]:
y_pred=np.expm1(y_pred)

In [37]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.527237610312857